# Seurat integration performed as in TOME

The following code is used to obtain the same representation as Qiu.
The code correponds to the code on github `ChengxiangQiu/tome_code/Section2_trajectory_Step1_connection.R` and 
`ChengxiangQiu/tome_code/help_code/help_code.R`
Qiu et al. used Seurat V3 in TOME, therefore Seurat V3 is used here as well.

#### seurat_object_XXX.rds can be downloaded from https://tome.gs.washington.edu/

## Import packages

In [1]:
library(Seurat)
library(future)
library(future.apply)

In [2]:
sessionInfo()

R version 4.1.3 (2022-03-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /mnt/home/icb/manuel.gander/miniconda3/envs/seurat4/lib/libopenblasp-r0.3.20.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=de_DE.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=de_DE.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=de_DE.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=de_DE.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] future.apply_1.9.0 future_1.26.1      Seurat_3.1.4      

loaded via a namespace (and not attached):
  [1] tsne_0.1-3.1        nlme_3.1-157        matrixStats_0.62.0 
  [4] RcppAnnoy_0.0.19    RColorBrewer_1.1-3  httr_1.4.3         
  [7] repr_1.1.4          numDeriv_2016.8-1.1 sc

In [3]:
file_path='/home/mgander/moscot-framework_reproducibility/data/mouse_temporal/RDS'

work_path = "/home/mgander/moscot-framework_reproducibility/data/mouse_temporal/Seurat_Representations"

time_point = paste0("E", c(3.5, 4.5, 5.25, 5.5, 6.25, seq(6.5, 8.25, 0.25), "8.5a", "8.5b", seq(9.5, 13.5, 1)))

In [4]:
###############################################
### Function: doing clustering using Seurat ###
###############################################


doClusterSeurat <- function(obj, nfeatures = 2500, resolution = 1, k.filter = 200, savePath = NULL, correctCC = FALSE, n_dim = 30, min.dist = 0.75){
    
    if(length(table(obj$group))!=1){
        
        obj.list <- SplitObject(object = obj, split.by = "group")
        
        for (i in 1:length(x = obj.list)) {
            obj.list[[i]] <- NormalizeData(object = obj.list[[i]], verbose = FALSE)
            obj.list[[i]] <- FindVariableFeatures(object = obj.list[[i]], 
                                                  selection.method = "vst", nfeatures = nfeatures, verbose = FALSE)
        }
        
        reference.list <- obj.list[names(table(obj$group))]
        obj.anchors <- FindIntegrationAnchors(object.list = reference.list, dims = 1:n_dim, k.filter = k.filter)
        obj.integrated <- IntegrateData(anchorset = obj.anchors, dims = 1:n_dim)
        
        # switch to integrated assay. The variable features of this assay are
        # automatically set during IntegrateData
        DefaultAssay(object = obj.integrated) <- "integrated"
        
        obj <- obj.integrated 
        
    } else {
        
        obj <- NormalizeData(obj, normalization.method = "LogNormalize", scale.factor = 10000)
        obj <- FindVariableFeatures(obj, selection.method = "vst", nfeatures = nfeatures)
        
    }
    
    if(correctCC == TRUE){
        obj <- ScaleData(object = obj, vars.to.regress = c("S.Score", "G2M.Score"), features = rownames(obj), verbose = FALSE)
    } else {
        obj <- ScaleData(object = obj, verbose = FALSE)
    }
    
    obj <- RunPCA(object = obj, npcs = n_dim, verbose = FALSE)
    obj <- FindNeighbors(object = obj, dims = 1:n_dim)
    obj <- FindClusters(object = obj, resolution = resolution)
    #  I added "n.components = 3" here, becasue in the pupblication Qiu et al repeatedly say they used 3 dims,
    # but the code they provided did not specify this and hence used the default 2-dim umap
    obj <- RunUMAP(object = obj, reduction = "pca", n.components = 3, dims = 1:n_dim, min.dist = min.dist)
    obj <- RunTSNE(object = obj, reduction = "pca", dims = 1:30)
    
    obj
}

# Calcualtion for all time pairs

This code is an exact copy of the code in https://github.com/ChengxiangQiu/tome_code, except for the amount of anchors used when filtering anchors. I had to adjust these due to error messages, likely due to the fact that I'm using different package versions then Qiu et al.

In [ ]:
for (i in 1:20) {
    time_1=time_point[i]
    print(time_1)
    time_2=time_point[i+1]


    if (time_2=='E7')
        {time_2='E7.0'}
    if (time_1=='E7')
        {time_1='E7.0'}

    obj_1 = readRDS(paste0(file_path, "/seurat_object_", time_1, ".rds"))
    obj_1$group = paste0(time_1, "_", obj_1$group)
    obj_2 = readRDS(paste0(file_path, "/seurat_object_", time_2, ".rds"))
    obj_2$group = paste0(time_2, "_", obj_2$group)
    obj = merge(obj_1, obj_2)
    
    
    if(!time_2 %in% paste0("E", c("8.5b", "9.5", "10.5", "11.5", "12.5", "13.5"))){
        
        ### These lines where not present in the TOME-code 
        ### (https://github.com/ChengxiangQiu/tome_code/blob/main/Section2_trajectory_Step1_connection.R)
        ### However, I encountered errors (if k.filter is bigger than the number of cells in a batch), that is
        ### why I had to use less cells as neighbors when filtering anchors
        if (i %in% c(1,  2, 3)){
            k.filter=50}
        else if (i %in% c(4,5,6)){
            k.filter=100}
        else {
            k.filter=200
        }
        
        obj.integrated = doClusterSeurat(obj, k.filter=k.filter)

    } else {

        obj.list <- SplitObject(obj, split.by = "group")
        obj.list <- future_lapply(X = obj.list, FUN = function(x) {
            x <- NormalizeData(x, verbose = FALSE)
            x <- FindVariableFeatures(x, verbose = FALSE)
        })

        features <- SelectIntegrationFeatures(object.list = obj.list)
        obj.list <- future_lapply(X = obj.list, FUN = function(x) {
            x <- ScaleData(x, features = features, verbose = FALSE)
            x <- RunPCA(x, features = features, verbose = FALSE)
        })

        anchors <- FindIntegrationAnchors(object.list = obj.list, reduction = "rpca", 
                                          dims = 1:50)
        obj.integrated <- IntegrateData(anchorset = anchors, dims = 1:50)

        obj.integrated <- ScaleData(obj.integrated, verbose = FALSE)
        obj.integrated <- RunPCA(obj.integrated, npcs = 30, verbose = FALSE)
        obj.integrated <- RunUMAP(obj.integrated, dims = 1:30, n.components = 3, min.dist = 0.75)

    }
    
    if (time_2=='E8')
        {time_2='E8.0'}
    if (time_1=='E8')
        {time_1='E8.0'}
    
    # Save the 30-dim PCA space and the 2d-umap for later use in TOME and Moscot
    umap = data.frame(Embeddings(object = obj.integrated, reduction = "umap"))
    write.csv(x=umap, file=paste0(work_path, "/", time_1, "_", time_2, "_umap3.csv"))
    saveRDS(umap, file=paste0(work_path, "/", time_1, "_", time_2, "_umap3.rds"))

    pca = data.frame(Embeddings(object = obj.integrated, reduction = "pca"))
    write.csv(x=pca, file=paste0(work_path, "/", time_1, "_", time_2, "_pca.csv"))
    saveRDS(pca, file=paste0(work_path, "/", time_1, "_", time_2, "_pca.rds"))
    }

[1] "E3.5"


Computing 2000 integration features

Scaling features for provided objects

Finding all pairwise anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 308 anchors

Filtering anchors

	Retained 267 anchors

Extracting within-dataset neighbors

Merging dataset 2 into 1

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Warning message:
“Adding a command log without an assay associated with it”
Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 157
Number of edges: 9264

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.1722
Number of communities: 4
Elapsed time: 0 seconds


Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”
13:15:25 UMAP embedding parameters a = 0.2734 b = 1.622

13:15:25 Read 157 rows and found 30 numeric columns

13:15:25 Using Annoy for neighbor search, n_neighbors = 30

13:15:25 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

13:15:25 Writing NN index file to temp file /tmp/RtmphY6ZJ0/file6d2b7b819e8

13:15:25 Searching Annoy index using 1 thread, search_k = 3000

13:15:25 Annoy recall = 100%

13:15:26 Commencing smooth kNN distance calibration using 1 thread

13:15:26 Initializing from 

[1] "E4.5"


...

Computing 2000 integration features

Scaling features for provided objects

Finding all pairwise anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 13536 anchors

Filtering anchors

	Retained 9092 anchors

Extracting within-dataset neighbors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 9460 anchors

Filtering anchors

	Retained 6791 anchors

Extracting within-dataset neighbors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 12463 anchors

Filtering anchors

	Retained 10505 anchors

Extracting within-dataset neighbors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 13046 anchors

Filtering anchors

	Retained 7312 anchors

Extracting within-dataset neighbors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 21976 anchors

Filtering anchors

	Retained 15727 anchors

Extracting within-dataset neighbors

Running CCA

Merging objects

Fin

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 31174
Number of edges: 1440294

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9085
Number of communities: 26
Elapsed time: 15 seconds


15:21:03 UMAP embedding parameters a = 0.2734 b = 1.622

15:21:03 Read 31174 rows and found 30 numeric columns

15:21:03 Using Annoy for neighbor search, n_neighbors = 30

15:21:03 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

15:21:14 Writing NN index file to temp file /tmp/RtmphY6ZJ0/file6d2b114274a2

15:21:14 Searching Annoy index using 1 thread, search_k = 3000

15:21:35 Annoy recall = 100%

15:21:37 Commencing smooth kNN distance calibration using 1 thread

15:21:40 Initializing from normalized Laplacian + noise

15:22:09 Commencing optimization for 200 epochs, with 1374640 positive edges

15:23:50 Optimization finished



[1] "E8"


Computing 2000 integration features

Scaling features for provided objects

Finding all pairwise anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 13488 anchors

Filtering anchors

	Retained 8378 anchors

Extracting within-dataset neighbors

Running CCA

Merging objects

Finding neighborhoods

